### IMPORTING LIBRARIES

In [24]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

### DOWNLOADING DATA

In [25]:
def returns(ticker_list):
    df_1 = pd.DataFrame()
    for ticker in tqdm(ticker_list):
        df = yf.download(ticker,start = '2024-01-01',auto_adjust=False)
        df['daily_ret'] = df['Adj Close'].pct_change()
        df_1[ticker] = df['daily_ret']
        df_1.dropna(inplace = True)
    return df_1
ticker_list = ['MSFT','AAPL','TSLA','AMZN','GOOGL','META']
n = len(ticker_list)
df = returns(ticker_list)

  0%|          | 0/6 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 6/6 [00:12<00:00,  2.15s/it]


In [26]:
df

,MSFT,AAPL,TSLA,AMZN,GOOGL,META
Date,,,,,,
2024-01-03,-0.000728,-0.007488,-0.040134,-0.009738,0.005428,-0.005256
2024-01-04,-0.007178,-0.012700,-0.002181,-0.026268,-0.018212,0.007693
2024-01-05,-0.000516,-0.004013,-0.001849,0.004634,-0.004839,0.013915
2024-01-08,0.018871,0.024175,0.012464,0.026577,0.022913,0.019065
2024-01-09,0.002936,-0.002263,-0.022832,0.015225,0.015197,-0.003429
...,...,...,...,...,...,...
2025-07-02,-0.001951,0.022231,0.049682,-0.002449,0.015924,-0.007856
2025-07-03,0.015781,0.005225,-0.000950,0.015869,0.004982,0.007624
2025-07-07,-0.002245,-0.016858,-0.067893,0.000269,-0.015262,-0.000918


### COVARIANCE MATRIX

In [27]:
sigma = df.cov()
Sigma = sigma.values
Sigma

array([[0.00021811, 0.00015047, 0.00026923, 0.00020844, 0.00016564,
        0.00022481],
       [0.00015047, 0.00034929, 0.00037387, 0.00019531, 0.00016611,
        0.00019491],
       [0.00026923, 0.00037387, 0.00185025, 0.00039523, 0.00034118,
        0.00037165],
       [0.00020844, 0.00019531, 0.00039523, 0.00039635, 0.000239  ,
        0.00033681],
       [0.00016564, 0.00016611, 0.00034118, 0.000239  , 0.00037476,
        0.00023273],
       [0.00022481, 0.00019491, 0.00037165, 0.00033681, 0.00023273,
        0.00059143]])

### SQP 

In [28]:
def portfolio_volatility(x, Sigma):
    return np.sqrt(x.T @ Sigma @ x)

def calculate_mcr(x, Sigma):
    portfolio_vol = portfolio_volatility(x, Sigma)
    mcr = (Sigma @ x) / portfolio_vol
    return mcr
    
def objective(x, Sigma):
    sum_squared_diff = 0
    MCR = calculate_mcr(x, Sigma) 
    for i in range(len(x)):
        for j in range(len(x)):
            diff = x[i] * MCR[i] - x[j] * MCR[j]
            sum_squared_diff += diff ** 2
    return sum_squared_diff


def constraint_eq(x):
    return np.sum(x) - 1

bounds = [(0, 1) for _ in range(n)]

x0 = np.ones(n) / n  

constraints = {'type': 'eq', 'fun': constraint_eq}

solution = minimize(objective, x0, args=(Sigma,), method='SLSQP', bounds=bounds, constraints=constraints)

print('Optimal solution:', solution.x)
print('Objective function value:', solution.fun)

Optimal solution: [0.18502944 0.18270173 0.09621165 0.17832873 0.18246099 0.17526746]
Objective function value: 9.760928570104167e-06


### ANALYSING PORTFOLIO

In [29]:
w = solution.x

# Calculating Annual Return
mean_daily_returns = df.mean()
expected_daily_return = np.dot(w, mean_daily_returns)
expected_annual_return = expected_daily_return*252
expected_annual_return_percent = expected_annual_return*100

# Calculating Annual Variance
cov_matrix_daily = df.cov()
portfolio_variance_daily = np.dot(w.T, np.dot(cov_matrix_daily, w))
portfolio_volatility_daily = np.sqrt(portfolio_variance_daily)
portfolio_variance_annual = portfolio_variance_daily * 252

# Calculating Annual Volatility
portfolio_volatility_annual = np.sqrt(portfolio_variance_annual)

# Print Results
print(f"Expected Annual Return: {expected_annual_return_percent:.2f}%")
print(f"Annual Variance: {portfolio_variance_annual:.6f}")
print(f"Annual Volatility (Std Dev): {portfolio_volatility_annual:.2f}")

Expected Annual Return: 29.73%
Annual Variance: 0.070000
Annual Volatility (Std Dev): 0.26
